# Optimizer Methods Testing

This is notebook for testing the workflow of the optimizer object in MDSAPT. Optimizer was developed to solve the issue of residues having an unbalanced spin when drawn directly from protein backbone. It does this over a few steps addressing the different issues. 

In [8]:
import os

import numpy as np

import MDAnalysis as mda

from pdbfixer import PDBFixer

from simtk.openmm.app import PDBFile

import nglview as nv

from MDSAPT.mdsapt.reader import InputReader

In [9]:
In = InputReader('test_input.yaml')

unv = mda.Universe(In.top_path, In.trj_path)

'ncpu' Error in trajectory settings


## 1. Loading the Residue

In this case ARG is the residue. Both the amino and carboxyl side of the residue are missing protons.

In [10]:
res = unv.select_atoms('resid 2')

nv.show_mdanalysis(res)

NGLWidget(max_frame=97)

## 2. Replacing missing Amino Protons
The file is loaded into PDBFixer from opennm and amino protons are reattached. This is the `_fix_amino()` method in Optimizer.

In [14]:
res.write('resid.pdb', file_format='PDB') # Saving residue
fixer = PDBFixer(filename='resid.pdb')
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingHydrogens(7) # Adding protons at pH value
PDBFile.writeFile(fixer.topology, fixer.positions, open('resid_fixed.pdb', 'w'))


res_fixed = mda.Universe('resid_fixed.pdb')
resid: mda.AtomGroup = res_fixed.select_atoms("resname *")
resid.guess_bonds()
nv.show_mdanalysis(resid)

NGLWidget()

## 3. Adding New Proton

The carbon on the carboxyl end of the residue only has three bonds when removed from the backbone. Rather than adding a hydroxyl group, a proton is added to the end as a 4th bond. This is done by the `_protonate_backbone()` method in Optimizer.

The new proton confidantes are obtained using the `_get_new_pos()` function by taking the opposite of the sum of the normalized vectors between the $C-\alpha C$ and the $C=O$ bond. That new vector is than normalized, multiplied by the new bond length, and added to the coordinates of the carbon. This gives the trigonal planar geometry expected in a SP2 carbon.

In [16]:
def _get_new_pos(backbone: mda.AtomGroup, length: float):
    c_pos = backbone.select_atoms('name C').positions[0]
    o_pos = backbone.select_atoms('name O').positions[0]
    a_pos = backbone.select_atoms('name CA').positions[0]
    o_pos = o_pos - c_pos  # Translate coords such that C in at origin
    a_pos = a_pos - c_pos
    o_norm = o_pos/np.linalg.norm(o_pos)
    a_norm = a_pos/np.linalg.norm(a_pos)
    h_pos = -(o_norm + a_norm)
    h_norm = h_pos/np.linalg.norm(h_pos)
    h_norm = (h_norm * length) + c_pos
    return h_norm

In [17]:
bkbone_fixed = mda.Universe('resid_fixed.pdb') # Load PDBRepair segid
bkbone_resid = bkbone_fixed.select_atoms("backbone") # Load backbone
ha = bkbone_fixed.select_atoms('name HA') # Remove proton on Alpha carbon
bkbone_resid = bkbone_resid - ha # Remove proton on Alpha carbon
carbon = bkbone_resid.select_atoms('name C') 
pos = carbon.atoms[0].position # Get postion of end carbon
h = mda.Universe.empty(n_atoms=bkbone_resid.n_atoms + 1, trajectory=True)
h.add_TopologyAttr("masses", [x for x in bkbone_resid.masses] + [1])
h.add_TopologyAttr("name", [x for x in bkbone_resid.names] + ['H'])
bkbone_pos = bkbone_resid.positions
h_pos = _get_new_pos(bkbone_fixed, 1.128)
h.atoms.positions = np.row_stack((bkbone_pos, h_pos))
nv.show_mdanalysis(h)


NGLWidget()

## 4. Optimizing New Bond

Psi4 is employed to optimize the length of the newly added C-H bond. Before this occurs the residue is converted into an rdkit molecule so that the spin multiplicity and formal charge can be easily computed for the psi4 input file.
Afterwards the coordinates are given to psi4 and the geometric optimization is run. This is done during the `_opt_geometry()` method of Optimizer.

In [22]:
from rdkit import Chem

from MDSAPT.mdsapt.optimizer import get_spin_multiplicity

from MDAnalysis.topology.guessers import guess_types, guess_atom_element
from MDAnalysis.converters.RDKit import atomgroup_to_mol

h.add_TopologyAttr('types', guess_types(h.atoms.names))
h.add_TopologyAttr('elements', [guess_atom_element(atom) for atom in h.atoms.names])
resid = h.select_atoms('all')

rd_mol = atomgroup_to_mol(resid)

In [24]:
import psi4

coords: str = f'{Chem.GetFormalCharge(rd_mol)} {get_spin_multiplicity(rd_mol)}'
freeze_list = ''
opt_settings: dict = {'reference': 'rhf'} # Psi4 settings

# Getting coords and settings
for n in range(len(h.atoms)):
    atom = h.atoms[n]
    coords += f"\n{atom.name[0]} {atom.position[0]} {atom.position[1]} {atom.position[1]}"
    if not atom.name == 'H':
        freeze_list += f"\n{n + 1} xyz" # Freezing bonds besides H
print(freeze_list)
print(coords)

mol = psi4.geometry(coords)



1 xyz
2 xyz
3 xyz
4 xyz
0 1
N 11.840999603271484 6.642000198364258 6.642000198364258
C 11.414999961853027 5.434000015258789 5.434000015258789
C 10.770999908447266 5.855000019073486 5.855000019073486
O 11.35200023651123 6.664000034332275 6.664000034332275
H 9.76501178741455 5.441152572631836 5.441152572631836


In [25]:
psi4.set_memory('8GB')
psi4.set_options(opt_settings)
psi4.optimize('scf/cc-pvdz', freeze_list=freeze_list, opt_coordinates='cartesian', molecule=mol)


  Memory set to   7.451 GiB by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on Alias-MBP
*** at Sat Dec 18 12:23:02 2021

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 4   entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 5   entry H          line    22 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
              

New linear angles



*** tstop() called on Alias-MBP at Sat Dec 18 12:23:09 2021
Module time:
	user time   =       0.40 seconds =       0.01 minutes
	system time =       0.03 seconds =       0.00 minutes
	total time  =          0 seconds =       0.00 minutes
Total time:
	user time   =      19.37 seconds =       0.32 minutes
	system time =       1.79 seconds =       0.03 minutes
	total time  =        580 seconds =       9.67 minutes

*** tstart() called on Alias-MBP
*** at Sat Dec 18 12:23:09 2021


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: cs
    Full point group: Cs

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X  

New linear angles


   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 4   entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 5   entry H          line    22 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 

  Reading orbitals from file /tmp/stdout.default.12032.180.npy, no projection.

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.010 GiB; user supplied 5.588 GiB. Using in-core AOs.

  ==> MemDFJK: Density-Fitted J/K Matrices <==

    J tasked:                   Yes
    K tasked:                   Yes
    wK tasked:                   No
    OpenMP threads:               1
    Memory [MiB]:              5722
    Algorithm:                 Cor

...

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 4   entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 5   entry H          line    22 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 

  Reading orbitals from file /tmp/stdout.default.12032.180.npy, no projection.

  ==> Integral Setup <==

  DFHelper Memory: AOs need 0.010 GiB; user supplied 5.588 GiB. Using in-core AOs.

  ==> MemDFJK: Density-Fitted J/K Matrices <==

    J tasked:                   Yes
    K tasked:                   Yes
    wK tasked:                   No
    OpenMP threads:               1
    Memory [MiB]:              5722
    Algorithm:                 Cor

New linear angles


   => Loading Basis Set <=

    Name: (CC-PVDZ AUX)
    Role: JKFIT
    Keyword: DF_BASIS_SCF
    atoms 1   entry N          line   171 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz-jkfit.gbs 
    atoms 2-3 entry C          line   121 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz-jkfit.gbs 
    atoms 4   entry O          line   221 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz-jkfit.gbs 
    atoms 5   entry H          line    51 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz-jkfit.gbs 

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line   168 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry C          line   138 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis/cc-pvdz.gbs 
    atoms 4   entry O          line   198 file /Users/alia/opt/anaconda3/envs/mdsapt2/share/psi4/basis

New linear angles


OptimizationConvergenceError: Could not converge geometry optimization in 50 iterations.